1. Environment Setup

In [5]:
!pip install -q \
  llama-index \
  EbookLib \
  html2text \
  gradio \
  llama-index-embeddings-huggingface \
  llama-index-llms-ollama

### Document Processing

1. Data Loading:

In [1]:
pwd

'/Users/sylviathsu/Documents/COS243/LocalAILibrarian'

In [2]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_files=["./library/TheTheoryThatWouldNotDie.pdf"])
documents = reader.load_data()

2. Text Chunking:

In [3]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

/opt/anaconda3/envs/gradio/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Parsing nodes: 100%|██████████| 392/392 [00:00<00:00, 2320.20it/s]


### Embedding Generation

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Define the embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [5]:
from llama_index.llms.ollama import Ollama
# Define LLM Model
llm = Ollama(model="phi3.5:3.8b-mini-instruct-q4_K_M")

In [6]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)


In [11]:
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex

# Set the global settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = nodes

# Create the vector index
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True)

AttributeError: 'list' object has no attribute 'callback_manager'

In [7]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)

ValueError: ServiceContext is deprecated. Use llama_index.settings.Settings instead, or pass in modules to local functions/methods/interfaces.
See the docs for updated usage/migration: 
https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/service_context_migration/

In [ ]:
library = "./library"
vector_index.storage_context.persist(persist_dir=library)
storage_context = StorageContext.from_defaults(persist_dir=library)

index = load_index_from_storage(storage_context, service_context=service_context)

### Query Engine

In [41]:
!pip show llama-index

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: llama-index
Version: 0.12.8
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: /opt/anaconda3/envs/gradio/lib/python3.11/site-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: 


In [34]:
query_engine = vector_index.as_query_engine(service_context=service_context)

ReadTimeout: timed out